In [1]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

/home/david.bertoin/anaconda3/envs/IRT_RL2/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/david.bertoin/anaconda3/envs/IRT_RL2/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import initializers;
from keras.optimizers import Adam

def compute_Q(env):
    
    # Model
    initializer = initializers.random_uniform(0, 0.1, seed=1)
    model = Sequential([
        #Dense(8, input_shape=(16,), use_bias=False),
        #Activation('relu'),
        Dense(4, use_bias=False, kernel_initializer=initializer),
        Activation('softmax'),
    ])

    model.compile(optimizer=Adam(lr=0.01),
                  loss='mse')
    
    
    # Set learning parameters
    y = .99
    e = 0.1
    num_episodes = 10000
    #create lists to contain total rewards and steps per episode
    jList = []
    rList = []

    for i in tqdm(range(num_episodes)):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Network
        while j < 99:
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            allQ = model.predict(np.identity(16)[s:s+1])
            a = np.argmax(allQ[0])
            if np.random.rand(1) < e:
                a = env.action_space.sample()
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a)
            #Obtain the Q' values by feeding the new state through our network
            Q1 = model.predict(np.identity(16)[s1:s1+1])
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a] = r + y*maxQ1
            #Train our network using target and predicted Q values
            model.fit(np.identity(16)[s:s+1], targetQ, verbose=False)
            
            rAll += r
            s = s1
            if d == True:
                #Reduce chance of random action as we train the model.
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)
    print ("Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%")
    return(model)

In [16]:
env = gym.make('FrozenLake-v0')
model = compute_Q(env)

100%|██████████| 10000/10000 [09:18<00:00, 17.89it/s]

Percent of succesful episodes: 0.2507%


In [17]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames, actions):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print("Timestep:%s" % (i + 1))
        print("State: %s" % frame['state'])
        print("Action:  %s" % actions[frame['action']])
        print("Reward:  %s" % frame['reward'])
        sleep(0.5)

In [22]:
def play_policy(env, model):
    s = env.reset()
    d = 0
    frames = []
    while not d:
        Q = model.predict(np.identity(16)[s:s+1])
        a = np.argmax(Q)
        s, r, d, _ = env.step(a)
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': s,
            'action': a,
            'reward': r
            })
    return frames        

In [23]:
frames = play_policy(env,model)
actions = {0: '\u2190', 1: '\u2193', 2: '\u2192', 3: '\u2191'}
print_frames(frames, actions)

  (Down)
SFFF
FHFH
FFFH
HFFG

Timestep:16
State: 15
Action:  ↓
Reward:  1.0
